In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!python --version

Python 3.7.16


In [14]:
import numpy as np
from statsmodels.stats.contingency_tables import cochrans_q

# 예시: 10명의 학생이 세 번의 시험(Test1, Test2, Test3)에서 합격(1) 또는 불합격(0)한 결과
# 귀무가설: 세 시험의 합격률은 모두 동일하다.
# 대립가설: 적어도 한 시험의 합격률은 다른 시험과 다르다.

data = np.array([
#   T1, T2, T3
    [1, 1, 0],  # 학생 1
    [1, 1, 1],  # 학생 2
    [0, 1, 1],  # 학생 3
    [0, 0, 0],  # 학생 4
    [1, 1, 0],  # 학생 5
    [0, 1, 0],  # 학생 6
    [1, 0, 0],  # 학생 7
    [1, 1, 1],  # 학생 8
    [0, 1, 0],  # 학생 9
    [0, 1, 1]   # 학생 10
])

# 코크란의 Q 검정 수행
result = cochrans_q(data)

print(f"Cochran's Q statistic: {result.statistic:.4f}") # 3.7143
print(f"P-value: {result.pvalue:.4f}")                  # 0.1561

# 결과 해석: "귀무가설 채택: 세 시험의 합격률은 통계적으로 차이가 없다고 볼 수 있습니다."
alpha = 0.05
if result.pvalue < alpha:
    print("귀무가설 기각: 세 시험의 합격률에는 통계적으로 유의미한 차이가 있습니다.")
else:
    print("귀무가설 채택: 세 시험의 합격률은 통계적으로 차이가 없다고 볼 수 있습니다.")

# 사후 분석 (Post-hoc Test)
# 만약 귀무가설이 기각되었다면, 어느 쌍(예: Test1-Test2, Test1-Test3, Test2-Test3)에서
# 차이가 나는지 확인하기 위해 사후 분석을 수행해야 합니다.
# 일반적으로 각 쌍에 대해 맥니마 검정(McNemar's Test)을 수행하고,
# 다중 비교에 따른 1종 오류 증가를 보정하기 위해 본페로니 교정(Bonferroni correction) 등을 적용합니다.

from statsmodels.stats.contingency_tables import mcnemar
import pandas as pd
import itertools

test_names = ['Test1', 'Test2', 'Test3']

# 사후 분석: 모든 쌍에 대해 McNemar 수행
results = []
for i, j in itertools.combinations(range(data.shape[1]), 2):
    # 2x2 교차표 생성
    table = pd.crosstab(data[:, i], data[:, j])
    # McNemar 검정 수행 (exact=False는 chi-square 근사)
    result = mcnemar(table, exact=False, correction=True)
    results.append({
        'comparison': f'{test_names[i]} vs {test_names[j]}',
        'statistic': result.statistic,
        'p_uncorrected': result.pvalue
    })

# Bonferroni 보정
p_values = [r['p_uncorrected'] for r in results]
p_adjusted = np.minimum(np.array(p_values) * len(p_values), 1.0)
for idx, r in enumerate(results):
    r['p_adjusted'] = p_adjusted[idx]
    r['reject'] = p_adjusted[idx] < alpha

# 결과 출력
posthoc_df = pd.DataFrame(results)
print("\nPost-hoc test (McNemar with Bonferroni correction):")
print(posthoc_df)

Cochran's Q statistic: 3.7143
P-value: 0.1561
귀무가설 채택: 세 시험의 합격률은 통계적으로 차이가 없다고 볼 수 있습니다.

Post-hoc test (McNemar with Bonferroni correction):
       comparison  statistic  p_uncorrected  p_adjusted  reject
0  Test1 vs Test2       0.80       0.371093    1.000000   False
1  Test1 vs Test3       0.00       1.000000    1.000000   False
2  Test2 vs Test3       2.25       0.133614    0.400843   False


In [15]:
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar


table = np.array([[20, 10],
                  [60, 10]])

# 맥니마 검정 수행
# b+c = 10+60 = 70 이므로 충분히 크지만, exact=True 사용이 일반적으로 더 안전합니다.
result = mcnemar(table, exact=True)

print(f"Statistic (b+c): {result.statistic}") # 10.0, exact=True일 경우, b와 c 중 작은 값
print(f"P-value: {result.pvalue:.4f}") # 0.0000

# 결과 해석: "귀무가설 기각: 약물 투여 전후의 완치율에는 통계적으로 유의미한 변화가 있습니다."
alpha = 0.05
if result.pvalue < alpha:
    print("귀무가설 기각: 약물 투여 전후의 완치율에는 통계적으로 유의미한 변화가 있습니다.")
else:
    print("귀무가설 채택: 약물 투여 전후의 완치율 변화는 유의미하지 않습니다.")

# 만약 exact=False (카이제곱 근사)를 사용한다면
result_chi2 = mcnemar(table, exact=False, correction=True)
print("\n--- Chi-squared approximation ---")
print(f"Chi-squared statistic: {result_chi2.statistic:.4f}")
print(f"P-value: {result_chi2.pvalue:.4f}")

Statistic (b+c): 10.0
P-value: 0.0000
귀무가설 기각: 약물 투여 전후의 완치율에는 통계적으로 유의미한 변화가 있습니다.

--- Chi-squared approximation ---
Chi-squared statistic: 34.3000
P-value: 0.0000
